packages installation

In [53]:
!pip install groq openai python-dotenv

settting up Api keys

In [ ]:
import os
API_KEY = input("Enter your Groq API key: ")
os.environ["GROQ_API_KEY"] = API_KEY

In [55]:
import os
import json
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from groq import Groq

# Initialize client
client = Groq(api_key=os.environ["GROQ_API_KEY"])
print("✅ Client initialized successfully!")

✅ Client initialized successfully!


Task 1

In [56]:
@dataclass
class ConversationConfig:
    max_turns: int = 10
    max_chars: int = 2000
    summarize_every: int = 3
    model: str = "llama-3.1-8b-instant"

class ConversationManager:
    def __init__(self, config: ConversationConfig):
        self.config = config
        self.conversation_history: List[Dict[str, str]] = []
        self.turn_count = 0

    def add_message(self, role: str, content: str):
        self.conversation_history.append({"role": role, "content": content})
        self.turn_count += 1

    def truncate_by_turns(self, max_turns: int) -> List[Dict[str, str]]:
        return self.conversation_history[-max_turns:]

    def truncate_by_chars(self, max_chars: int) -> List[Dict[str, str]]:
        truncated_history = []
        current_length = 0

        for message in reversed(self.conversation_history):
            message_length = len(json.dumps(message))
            if current_length + message_length <= max_chars:
                truncated_history.insert(0, message)
                current_length += message_length
            else:
                break
        return truncated_history

    def summarize_conversation(self) -> str:
        summary_prompt = [
            {"role": "system", "content": "Summarize this conversation concisely."},
            {"role": "user", "content": f"Summarize:\n{json.dumps(self.conversation_history, indent=2)}"}
        ]

        try:
            response = client.chat.completions.create(
                model=self.config.model,
                messages=summary_prompt,
                max_tokens=300,
                temperature=0.1
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Summary failed: {str(e)}"

    def get_truncated_history(self) -> List[Dict[str, str]]:
        truncated = self.truncate_by_turns(self.config.max_turns)
        history_json = json.dumps(truncated)
        if len(history_json) > self.config.max_chars:
            truncated = self.truncate_by_chars(self.config.max_chars)
        return truncated

    def should_summarize(self) -> bool:
        return self.turn_count % self.config.summarize_every == 0 and self.turn_count > 0

    def generate_response(self, user_message: str) -> str:
        self.add_message("user", user_message)

        if self.should_summarize():
            summary = self.summarize_conversation()
            self.conversation_history = [
                {"role": "system", "content": f"Previous summary: {summary}"}
            ]
            self.turn_count = 1

        current_history = self.get_truncated_history()

        if not any(msg["role"] == "system" for msg in current_history):
            current_history.insert(0, {
                "role": "system",
                "content": "You are a helpful assistant."
            })

        try:
            response = client.chat.completions.create(
                model=self.config.model,
                messages=current_history,
                max_tokens=500,
                temperature=0.7
            )

            assistant_response = response.choices[0].message.content
            self.add_message("assistant", assistant_response)
            return assistant_response

        except Exception as e:
            return f"Error: {str(e)}"

    def display_conversation(self):
        print("=" * 50)
        print("CURRENT CONVERSATION HISTORY:")
        print("=" * 50)
        for i, message in enumerate(self.conversation_history, 1):
            print(f"{i}. {message['role'].upper()}: {message['content']}")
        print("=" * 50)
        print(f"Total turns: {self.turn_count}")
        print("=" * 50)

In [57]:
def test_conversation_manager():
    print("Testing Conversation Manager with Summarization")
    print("=" * 60)

    #model
    config = ConversationConfig(
        max_turns=4,
        max_chars=1000,
        summarize_every=2,
        model="llama-3.1-8b-instant"
    )

    manager = ConversationManager(config)

    # conversation samples for testing
    conversation_samples = [
        "Hello! Can you help me learn about AI?",
        "What is machine learning?",
        "Tell me about neural networks",
        "How are they used in real world?",
        "What programming languages are used for AI?",
        "Thanks for the information!"
    ]

    for i, message in enumerate(conversation_samples, 1):
        print(f"\nTurn {i}: User: {message}")
        response = manager.generate_response(message)
        print(f"Assistant: {response}")

        if manager.should_summarize():
            print("\n" + "="*40)
            print("SUMMARIZATION TRIGGERED!")
            print("="*40)
            manager.display_conversation()

    #final state
    print("\n" + "="*60)
    print("FINAL CONVERSATION STATE:")
    print("="*60)
    manager.display_conversation()

# Run test
test_conversation_manager()

Testing Conversation Manager with Summarization

Turn 1: User: Hello! Can you help me learn about AI?
Assistant: I'd be happy to help you learn about AI.

Artificial Intelligence (AI) is a rapidly growing field that involves the development of computer systems that can perform tasks that would typically require human intelligence, such as learning, problem-solving, decision-making, and perception.

There are several key areas of focus in AI:

1. **Machine Learning (ML)**: A subset of AI that involves training algorithms to learn from data and improve their performance over time.
2. **Deep Learning (DL)**: A type of ML that uses neural networks with multiple layers to learn complex patterns in data.
3. **Natural Language Processing (NLP)**: A field of AI that deals with the interaction between computers and humans in natural language.
4. **Computer Vision**: A field of AI that deals with the interpretation and understanding of visual data from images and videos.
5. **Robotics**: A field

Task 2

In [58]:
class InformationExtractor:
    def __init__(self, model: str = "llama-3.1-8b-instant"):
        self.model = model
        self.schema = {
            "name": "extract_user_information",
            "description": "Extract personal information from user messages",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Full name of the person"
                    },
                    "email": {
                        "type": "string",
                        "description": "Email address"
                    },
                    "phone": {
                        "type": "string",
                        "description": "Phone number with country code"
                    },
                    "location": {
                        "type": "string",
                        "description": "City and country"
                    },
                    "age": {
                        "type": "integer",
                        "description": "Age in years"
                    }
                },
                "required": ["name", "email", "phone", "location", "age"]
            }
        }

    def extract_information(self, user_message: str) -> Dict[str, Any]:
        """Extract structured information using function calling"""
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": "You are an information extraction assistant. Extract personal details from user messages."
                    },
                    {
                        "role": "user",
                        "content": user_message
                    }
                ],
                tools=[{
                    "type": "function",
                    "function": self.schema
                }],
                tool_choice={"type": "function", "function": {"name": "extract_user_information"}},
                max_tokens=500,
                temperature=0.1
            )

            # Extracting the function call response
            tool_call = response.choices[0].message.tool_calls[0]
            arguments = json.loads(tool_call.function.arguments)

            return arguments

        except Exception as e:
            return {"error": f"Extraction failed: {str(e)}"}

    def validate_extraction(self, extracted_data: Dict[str, Any]) -> Dict[str, Any]:
        """Validate extracted data against schema"""
        validation_result = {
            "is_valid": True,
            "errors": [],
            "missing_fields": []
        }

        # Check required fields
        required_fields = self.schema["parameters"]["required"]
        for field in required_fields:
            if field not in extracted_data or not extracted_data[field]:
                validation_result["is_valid"] = False
                validation_result["missing_fields"].append(field)

        # Validate data types
        properties = self.schema["parameters"]["properties"]
        for field, value in extracted_data.items():
            if field in properties:
                expected_type = properties[field]["type"]
                if expected_type == "integer" and not isinstance(value, int):
                    validation_result["is_valid"] = False
                    validation_result["errors"].append(f"{field} should be integer")
                elif expected_type == "string" and not isinstance(value, str):
                    validation_result["is_valid"] = False
                    validation_result["errors"].append(f"{field} should be string")

        return validation_result


In [59]:
# Run test function
def test_information_extraction():
    print("\n" + "="*60)
    print("Testing Information Extraction")
    print("="*60)

    extractor = InformationExtractor()

    sample_chats = [
        "Hi, my name is John Doe. I'm 25 years old and I live in New York, USA. My email is john.doe@email.com and phone is +1-555-0123.",
        "I'm Sarah Smith from London, UK. You can reach me at sarah.smith@company.co.uk or call me at +44-20-7946-0958. I'm 30 years old.",
        "My details: Name - Mike Johnson, Age - 35, Location - Tokyo, Japan, Email - mike.j@example.com, Phone - +81-3-1234-5678"
    ]

    for i, chat in enumerate(sample_chats, 1):
        print(f"\nSample {i}: {chat}")
        print("-" * 40)

        extracted_data = extractor.extract_information(chat)
        validation = extractor.validate_extraction(extracted_data)

        print("Extracted Information:")
        for key, value in extracted_data.items():
            print(f"  {key}: {value}")

        print(f"\nValidation: {'✅ Valid' if validation['is_valid'] else '❌ Invalid'}")
        if validation['errors']:
            print(f"Errors: {validation['errors']}")
        if validation['missing_fields']:
            print(f"Missing: {validation['missing_fields']}")

        print("-" * 40)

test_information_extraction()


Testing Information Extraction

Sample 1: Hi, my name is John Doe. I'm 25 years old and I live in New York, USA. My email is john.doe@email.com and phone is +1-555-0123.
----------------------------------------
Extracted Information:
  age: 25
  email: john.doe@email.com
  location: New York, USA
  name: John Doe
  phone: +1-555-0123

Validation: ✅ Valid
----------------------------------------

Sample 2: I'm Sarah Smith from London, UK. You can reach me at sarah.smith@company.co.uk or call me at +44-20-7946-0958. I'm 30 years old.
----------------------------------------
Extracted Information:
  age: 30
  email: sarah.smith@company.co.uk
  location: London, UK
  name: Sarah Smith
  phone: +44-20-7946-0958

Validation: ✅ Valid
----------------------------------------

Sample 3: My details: Name - Mike Johnson, Age - 35, Location - Tokyo, Japan, Email - mike.j@example.com, Phone - +81-3-1234-5678
----------------------------------------
Extracted Information:
  age: 35
  email: mike.j@

In [60]:
# Running complete demonstration
def main():
    print("GROQ API Assignment - Conversation Management & Classification")
    print("=" * 70)

    # Task 1
    print("\nTASK 1: CONVERSATION MANAGEMENT WITH SUMMARIZATION")
    print("=" * 70)
    test_conversation_manager()

    # Task 2
    print("\nTASK 2: JSON SCHEMA CLASSIFICATION & INFORMATION EXTRACTION")
    print("=" * 70)
    test_information_extraction()

    print("\n" + "=" * 70)
    print("ASSIGNMENT COMPLETE!")
    print("=" * 70)

main()

GROQ API Assignment - Conversation Management & Classification

TASK 1: CONVERSATION MANAGEMENT WITH SUMMARIZATION
Testing Conversation Manager with Summarization

Turn 1: User: Hello! Can you help me learn about AI?
Assistant: I'd be happy to help you learn about AI (Artificial Intelligence).

AI is a broad field that involves creating machines or computer programs that can perform tasks that typically require human intelligence, such as:

1. **Learning**: AI systems can learn from data, experiences, and interactions.
2. **Reasoning**: AI systems can make decisions based on logic, rules, and evidence.
3. **Problem-solving**: AI systems can identify problems, analyze them, and come up with solutions.
4. **Perception**: AI systems can interpret and understand data from sensors, images, and sounds.

There are several types of AI, including:

1. **Narrow or Weak AI**: Designed to perform a specific task, such as facial recognition, language translation, or playing chess.
2. **General or S